In [2]:
import glob
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import project_functions


cars = glob.glob('./vehicles/KITTI_extracted/*.png') + glob.glob('./vehicles/GTI_Far/*.png') + glob.glob('./vehicles/GTI_Left/*.png') + glob.glob('./vehicles/GTI_Right/*.png') + glob.glob('./vehicles/GTI_MiddleClose/*.png')
non_cars = glob.glob('./non-vehicles/GTI/*.png') + glob.glob('./non-vehicles/Extras/*.png')


In [3]:
non_car_features = project_functions.extract_features(non_cars, color_space='YCrCb', hog_channel='ALL')
car_features = project_functions.extract_features(cars, color_space='YCrCb', hog_channel='ALL')

/home/iyed/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [4]:
X = np.vstack((car_features, non_car_features)).astype(np.float64)                        
X_scaler = StandardScaler().fit(X)

In [5]:
scaled_X = X_scaler.transform(X)

In [6]:
y = np.hstack((np.ones(len(car_features)), np.zeros(len(non_car_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

In [7]:
svc = LinearSVC()
svc.fit(X_train, y_train)
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample

n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])

Test Accuracy of SVC =  0.9924
My SVC predicts:  [ 0.  1.  1.  0.  1.  0.  1.  0.  0.  1.]
For these 10 labels:  [ 0.  1.  1.  0.  1.  0.  1.  0.  0.  1.]


In [8]:
import pickle


dist_pickle = {}

dist_pickle['svc'] = svc
dist_pickle["scaler"] = X_scaler
dist_pickle["orient"] = 9
dist_pickle["pix_per_cell"] = 8
dist_pickle["cell_per_block"] = 2
dist_pickle["spatial_size"] = (32, 32)
dist_pickle["hist_bins"] = 32

pickle.dump(dist_pickle, open('classifier.p', 'wb'))